## Sources:

Zip code link: https://gis-mdc.opendata.arcgis.com/datasets/fee863cb3da0417fa8b5aaf6b671f8a7/explore

## Imports

In [17]:
import pandas as pd
import geopandas as gpd
import altair as alt
import json

## Data read-in

In [47]:
df = pd.read_csv('CoreLogicData - HPI Gain by Zip Code in Miami Dade, FL.csv')

gdf = gpd.read_file('Miami_Dade_Zip_Code.geojson', geometry='geometry')

In [49]:
# Now, you can check for valid geometries
gdf_clean = gdf[gdf.geometry.notnull()]

In [52]:
# Note: gpd.GeoDataFrame does have the 'is_valid' attribute, but it applies to the 'geometry' Series
gdf_valid = gdf_clean[gdf_clean.geometry.is_valid]

## Data Prepare

In [54]:
df['5 Year HPI Change'] = df['5 Year HPI Change'].str.rstrip('%').astype(float) / 100.0

In [55]:
df = df.rename(columns={'Zip Code':'zip_code','5 Year HPI Change':'percentage'})

In [61]:
merged_df = pd.merge(df, gdf_valid, left_on='zip_code',right_on='ZIP')
len(merged_df)

21

In [62]:
gdf_clean = merged_df[merged_df.geometry.notnull()]
len(gdf_clean)

21

In [63]:
merged_df['geometry'].isna()

0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
15    False
16    False
17    False
18    False
19    False
20    False
Name: geometry, dtype: bool

In [65]:
# Assuming gdf is your GeoDataFrame
# Make sure your GeoDataFrame has a 'percentage' column and a 'ZIP' column

# Center coordinates for Miami-Dade County
latitude = 25.774265
longitude = -80.193658

# Initialize Folium Map at a central point
m = folium.Map(location=[latitude, longitude], zoom_start=11, tiles='cartodbpositron')

# Function to assign color based on 'percentage'
def style_function(feature):
    return {
        'fillColor': '#green' if feature['properties']['percentage'] > 50 else '#red',
        'color': 'black',
        'weight': 1.5,
        'fillOpacity': 0.5
    }

# Add the GeoDataFrame as a GeoJson layer
folium.GeoJson(
    gdf_clean,
    tooltip=folium.GeoJsonTooltip(fields=['ZIP', 'percentage'],
                                  aliases=['ZIP Code:', 'Percentage:'],
                                  localize=True)
).add_to(m)

# Add a LayerControl and display
folium.LayerControl().add_to(m)

# Save or display map
# m.save('choropleth_map.html')  # Saves the map to an HTML file
# or display it directly in a Jupyter notebook with just 'm'
m

ValueError: Cannot render objects with any missing geometries:     zip_code  2018\n December  2019\n December  2020\n December  \
0      33125         226.0082         236.5720         257.7418   
1      33126         287.1707         295.0787         310.6945   
2      33127         259.9927         269.5568         334.2315   
3      33129         228.4923         234.9700         248.5591   
4      33130         222.4877         237.1577         245.3386   
5      33131         226.2215         228.0842         229.0576   
6      33132         220.1432         219.7303         228.6706   
7      33133         236.3068         248.5824         269.8200   
8      33134         250.3498         258.1528         276.0575   
9      33137         221.1702         229.3062         242.4450   
10     33138         334.8305         359.8006         381.6110   
11     33139         285.5352         295.7397         317.1138   
12     33140         286.2844         294.4096         307.8340   